Rhizome Dataframes: 
- 1 x Artist DF based on rdf information + manual additions 
- 1 x Artworks DF based on rdf information + manual checks 
- 1 x Artworks DF w/ additional information (gender, nationality) from Artist DF for visualization 

In [1]:
import csv
import pandas as pd
import pandas_profiling as pp
path = '/Users/laurentfintoni/Desktop/University/COURSE DOCS/YEAR 2/EPUB/PROJECT/EPDS/Rhizome_data/'

In [11]:
#create a df for artists, pickle 

rhz_artists= pd.read_csv('./Rhizome_data/artists_complete.csv', dtype='string', encoding = "ISO-8859-1")
#set empty fields w/ missing
rhz_artists['artistLabel'] = rhz_artists['artistLabel'].fillna('')
rhz_artists['Nationality'] = rhz_artists['Nationality'].fillna('missing')
rhz_artists['Nationality Guessed'] = rhz_artists['Nationality Guessed'].fillna('missing')
rhz_artists['Gender'] = rhz_artists['Gender'].fillna('missing')
rhz_artists['Gender Guessed'] = rhz_artists['Gender Guessed'].fillna('missing')
rhz_artists['Birth'] = rhz_artists['Birth'].fillna('0').astype('int')
rhz_artists['Death'] = rhz_artists['Death'].fillna('0').astype('int')
rhz_artists['Wiki QID'] = rhz_artists['Wiki QID'].fillna('')
rhz_artists['ULAN'] = rhz_artists['ULAN'].fillna('')
#set collective fields to empty not missing 
rhz_artists['collectiveLabel'] = rhz_artists['collectiveLabel'].fillna('')
rhz_artists['collectivePage'] = rhz_artists['collectivePage'].fillna('')
#create and populate ID field to cross reference with artworks 
rhz_artists['ID'] = range(1, 1+len(rhz_artists))
#drop the notes column 
rhz_artists = rhz_artists.drop('Notes', axis=1)
#rename columns 
rhz_artists.rename(columns={'artistLabel': 'Artist', 'artistPage': 'Artist URL', 'collectiveLabel': 'Collective', 'collectivePage': 'Collective URL', 'Wiki QID': 'WikiQID', 'Nationality Guessed': 'nationalityGuessed', 'Gender Guessed': 'genderGuessed', 'Wiki QID': 'wikiQID'}, inplace=True)
rhz_artists.to_pickle(path+'rhizome_artists.pkl')

In [12]:
#create a df for artworks, pickle 

rhz_artworks= pd.read_csv('./Rhizome_data/artwork_complete.csv', dtype='string', encoding = "ISO-8859-1")
#set empty fields w/ missing or empty strings 
rhz_artworks['accession'] = rhz_artworks['accession'].fillna('0')
rhz_artworks.loc[:,'accession'] = rhz_artworks['accession'].where((rhz_artworks['accession'].str.len() <= 4), rhz_artworks['accession'].str[0:4])
rhz_artworks['inception'] = rhz_artworks['inception'].fillna('0')
rhz_artworks.loc[:,'inception'] = rhz_artworks['inception'].where((rhz_artworks['inception'].str.len() <= 4), rhz_artworks['inception'].str[0:4])
url_columns = ['summary_url', 'summary_url_2', 'description_url', 'description_url_2', 'statement_url']
rhz_artworks[url_columns] = rhz_artworks[url_columns].fillna('')
#rename columns 
rhz_artworks.rename(columns={'artwork_label': 'Title', 'artist_label': 'Artist', 'accession': 'dateAcquired', 'inception': 'dateCreated', 'artwork_page': 'URL'}, inplace=True)
#auto populate ID based on artist df 
for index, row in rhz_artworks.iterrows():
    ids = list()
    artist = getattr(row, 'Artist')
    artist = artist.split(', ')
    artists = rhz_artists[['Artist', 'ID']]
    artists = dict(list(zip(artists.Artist, artists.ID)))
    collectives = rhz_artists[['Collective', 'ID']]
    collectives = dict(list(zip(collectives.Collective, collectives.ID)))

    for person in artist:
        if person in artists:
            ids.append(str(artists[person]))
        elif person in collectives:
            ids.append(str(collectives[person]))
        else:
            ids.append('missing')
    string = ', '.join(ids)

    rhz_artworks.at[index, 'ID'] = string
rhz_artworks.to_pickle(path+'rhizome_artworks.pkl')

In [14]:
#create a second df for artworks w/ additional artist info for visualisation 

rhz_artworks_extra = rhz_artworks.copy()
rhz_artworks_extra.drop(['summary_url', 'summary_url_2', 'description_url', 'description_url_2', 'statement_url'], inplace=True, axis=1)

#add nationality 
for index, row in rhz_artworks_extra.iterrows():
    nationality = list()
    artist = getattr(row, 'Artist')
    artist = artist.split(', ')
    artists = rhz_artists[['Artist', 'Nationality']]
    artists = dict(list(zip(artists.Artist, artists.Nationality)))
    collectives = rhz_artists[['Collective', 'Nationality']]
    collectives = dict(list(zip(collectives.Collective, collectives.Nationality)))

    for person in artist:
        if person in artists:
            nationality.append(str(artists[person]))
        elif person in collectives:
            nationality.append(str(collectives[person]))
        else:
            nationality.append('missing')
    string_nationality = ', '.join(nationality)

    rhz_artworks_extra.at[index, 'Nationality'] = string_nationality
    
#add guessed nationality 
for index, row in rhz_artworks_extra.iterrows():
    nationality_2 = list()
    artist = getattr(row, 'Artist')
    artist = artist.split(', ')
    artists = rhz_artists[['Artist', 'nationalityGuessed']]
    artists = dict(list(zip(artists.Artist, artists.nationalityGuessed)))
    collectives = rhz_artists[['Collective', 'nationalityGuessed']]
    collectives = dict(list(zip(collectives.Collective, collectives.nationalityGuessed)))

    for person in artist:
        if person in artists:
            nationality_2.append(str(artists[person]))
        elif person in collectives:
            nationality_2.append(str(collectives[person]))
        else:
            nationality_2.append('missing')
    string_nationality_2 = ', '.join(nationality_2)

    rhz_artworks_extra.at[index, 'nationalityGuessed'] = string_nationality_2
    
#add gender 
for index, row in rhz_artworks_extra.iterrows():
    gender = list()
    artist = getattr(row, 'Artist')
    artist = artist.split(', ')
    artists = rhz_artists[['Artist', 'Gender']]
    artists = dict(list(zip(artists.Artist, artists.Gender)))
    collectives = rhz_artists[['Collective', 'Gender']]
    collectives = dict(list(zip(collectives.Collective, collectives.Gender)))

    for person in artist:
        if person in artists:
            gender.append(str(artists[person]))
        elif person in collectives:
            gender.append(str(collectives[person]))
        else:
            gender.append('missing')
    string_gender = ', '.join(gender)

    rhz_artworks_extra.at[index, 'Gender'] = string_gender

#add guessed gender 
for index, row in rhz_artworks_extra.iterrows():
    gender_2 = list()
    artist = getattr(row, 'Artist')
    artist = artist.split(', ')
    artists = rhz_artists[['Artist', 'genderGuessed']]
    artists = dict(list(zip(artists.Artist, artists.genderGuessed)))
    collectives = rhz_artists[['Collective', 'genderGuessed']]
    collectives = dict(list(zip(collectives.Collective, collectives.genderGuessed)))

    for person in artist:
        if person in artists:
            gender_2.append(str(artists[person]))
        elif person in collectives:
            gender_2.append(str(collectives[person]))
        else:
            gender_2.append('missing')
    string_gender_2 = ', '.join(gender_2)

    rhz_artworks_extra.at[index, 'genderGuessed'] = string_gender_2
rhz_artworks_extra.to_pickle(path+'rhizome_artworks.pkl')

Reports 

In [ ]:
pp.ProfileReport(rhz_artists)